In [6]:
#!pip install keras-tuner
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from pathlib import Path


#  Import and read the charity_data.csv.
beetus_path = Path("diabetes_binary_5050split_health_indicators_BRFSS2015.csv")
beetus_df = pd.read_csv(beetus_path)
beetus_df.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0


In [7]:
y = beetus_df['Diabetes_binary'].values
X = beetus_df.drop(['Diabetes_binary'],1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\lpfro\AppData\Local\Temp\ipykernel_26564\4085878295.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = beetus_df.drop(['Diabetes_binary'],1).values


In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
num_input_features = len(X_train[0])

In [10]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    activation = hp.Choice('activation',['relu','tanh','softmax','sigmoid'])
    
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=num_input_features))
    for i in range(hp.Int('num_layers', 1,6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    nn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
    
    nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return nn_model

In [11]:
tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=20,
    hyperband_iterations=2)


Reloading Tuner from .\untitled_project\tuner0.json


In [12]:
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

In [13]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 9,
 'num_layers': 4,
 'units0': 9,
 'units1': 1,
 'units2': 3,
 'units3': 9,
 'units4': 1,
 'units5': 1,
 'tuner/epochs': 7,
 'tuner/initial_epoch': 3,
 'tuner/bracket': 2,
 'tuner/round': 1,
 'tuner/trial_id': '0036'}

In [14]:
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")





553/553 - 1s - loss: 0.5055 - accuracy: 0.7541 - 1s/epoch - 2ms/step
Loss: 0.5055218935012817, Accuracy: 0.7541447281837463


In [15]:
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

553/553 - 1s - loss: 0.5055 - accuracy: 0.7541 - 804ms/epoch - 1ms/step
Loss: 0.5055218935012817, Accuracy: 0.7541447281837463
553/553 - 1s - loss: 0.5019 - accuracy: 0.7540 - 744ms/epoch - 1ms/step
Loss: 0.5019425749778748, Accuracy: 0.753974974155426
553/553 - 1s - loss: 0.5040 - accuracy: 0.7538 - 870ms/epoch - 2ms/step
Loss: 0.503968358039856, Accuracy: 0.7538052201271057


In [16]:
testing_report = classification_report(y_test, X_test_scaled)
print(testing_report)

NameError: name 'classification_report' is not defined